Tutorials on 3-D tensor of generating cellxgenexembedding representation
===

embed_dim = 1 x raw_gene + 5 x hop_neighbors + (embed_dim - 5) x random_walk

In [10]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
import warnings
warnings.filterwarnings("ignore")


tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


卷积层

In [69]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_remaining_self_loops, spmm
class GeneRep(MessagePassing):
    def __init__(self, add_self_loop=True):
        super().__init__(flow='source_to_target', aggr='mean')
        self.n_hops = 3
        self.add_self_loop = add_self_loop
        
    def forward(self, x, edge_index):
        if self.add_self_loop:
            edge_index = add_remaining_self_loops(edge_index)[0]
        emb = []
        emb.append(x)
        for i in range(self.n_hops):
            emb.append(self.propagate(edge_index, x=emb[i]))
        return emb
    def message(self, x_j):
        print('1')
        return x_j

    # def message_and_aggregate(self, adj_t, x):
    #     print(adj_t)
    #     return spmm(adj_t, x, reduce=self.aggr)

In [19]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long).T
x = torch.tensor([[-1, 0], [0, 1], [1, 1]], dtype=torch.float)
x, edge_index

(tensor([[-1.,  0.],
         [ 0.,  1.],
         [ 1.,  1.]]),
 tensor([[0, 1, 1, 2],
         [1, 0, 2, 1]]))

In [70]:
model = GeneRep()
emb=model(x, edge_index)

1
1
1


In [57]:
emb

[tensor([[-1.,  0.],
         [ 0.,  1.],
         [ 1.,  1.]]),
 tensor([[-0.5000,  0.5000],
         [ 0.0000,  0.6667],
         [ 0.5000,  1.0000]]),
 tensor([[-0.2500,  0.5833],
         [ 0.0000,  0.7222],
         [ 0.2500,  0.8333]]),
 tensor([[-0.1250,  0.6528],
         [ 0.0000,  0.7130],
         [ 0.1250,  0.7778]])]

In [58]:
torch.cat(emb).view(4,3,2).permute(1,2,0)

tensor([[[-1.0000, -0.5000, -0.2500, -0.1250],
         [ 0.0000,  0.5000,  0.5833,  0.6528]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.6667,  0.7222,  0.7130]],

        [[ 1.0000,  0.5000,  0.2500,  0.1250],
         [ 1.0000,  1.0000,  0.8333,  0.7778]]])

随机游走

In [1]:
import networkx as nx # 图数据挖掘
import numpy as np # 数据分析
import random # 随机数

# 数据可视化
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签  
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号b

In [2]:
# 《悲惨世界》人物数据集
G = nx.les_miserables_graph()

In [3]:
from node2vec import Node2Vec

In [11]:
node2vec = Node2Vec(G, 
                    dimensions=32,  # 嵌入维度
                    p=1,            # 回家参数
                    q=3,          # 外出参数
                    walk_length=10, # 随机游走最大长度
                    num_walks=1,  # 每个节点作为起始节点生成的随机游走个数
                    workers=4       # 并行线程数
                   )

Computing transition probabilities:   0%|          | 0/77 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 1/1 [00:00<00:00, 590.41it/s]
Generating walks (CPU: 2): 0it [00:00, ?it/s]
Generating walks (CPU: 3): 0it [00:00, ?it/s]
Generating walks (CPU: 4): 0it [00:00, ?it/s]


In [12]:
node2vec.num_walks

1

In [13]:
len(node2vec.walks)

77

PyG实现

In [20]:
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
data = Data(edge_index=edge_index, num_nodes=3)
G = to_networkx(data)

In [21]:
from node2vec import Node2Vec
node2vec = Node2Vec(G, 
                    dimensions=1,  # 嵌入维度
                    p=1,            # 回家参数
                    q=3,          # 外出参数
                    walk_length=10, # 随机游走最大长度
                    num_walks=1,  # 每个节点作为起始节点生成的随机游走个数
                    workers=4       # 并行线程数
                   )

Computing transition probabilities:   0%|          | 0/3 [00:00<?, ?it/s]

Generating walks (CPU: 2): 0it [00:00, ?it/s]/1 [00:00<?, ?it/s]
Generating walks (CPU: 3): 0it [00:00, ?it/s]
Generating walks (CPU: 1): 100%|██████████| 1/1 [00:00<00:00, 10645.44it/s]
Generating walks (CPU: 4): 0it [00:00, ?it/s]


In [24]:
edge_index

tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

In [27]:
G.nodes()

NodeView((0, 1, 2))

In [35]:
node2vec.walks

[['2', '1', '0', '1', '0', '1', '0', '1', '2', '1'],
 ['0', '1', '0', '1', '2', '1', '2', '1', '0', '1'],
 ['1', '2', '1', '0', '1', '2', '1', '0', '1', '0']]

In [40]:
walks_int = [[int(node) for node in walk] for walk in node2vec.walks]
walks_int

[[2, 1, 0, 1, 0, 1, 0, 1, 2, 1],
 [0, 1, 0, 1, 2, 1, 2, 1, 0, 1],
 [1, 2, 1, 0, 1, 2, 1, 0, 1, 0]]

In [52]:
edge = [[],[]]
for i in range(len(walks_int)):
    for j in range(10):
        edge[0].append(walks_int[i][j])
        edge[1].append(i)
np.array(edge)

array([[2, 1, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 1, 2,
        1, 0, 1, 2, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2]])